# Reaction essentiality

In [5]:
import pandas as pd
from cobra.io.json import load_json_model
import numpy as np
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis import single_reaction_deletion
import math
from tqdm import tqdm

In [19]:
def rxn_essential(model):
    df_res_rxn = single_reaction_deletion(model, processes=10)
    df_essential=df_res_rxn[df_res_rxn["status"]!="optimal"] # remove non essential
    df_essential["rxn_ids"]=df_essential.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x) # remove {} around ids

    def change_bounds(rxn_id):
        if rxn_id in model.reactions:
            rxn= model.reactions.get_by_id(rxn_id)
            lb,ub= rxn.lower_bound, rxn.upper_bound
            M=1000

            if ub <= 0 and lb < 0:
                rxn_bounds=(-M, 0.0)
            elif lb >= 0 and ub > 0:
                rxn_bounds=(0.0, +M)
            else:
                rxn_bounds=(-M, +M)
        
            
            rxn.bounds=rxn_bounds
        else:
            print("error reaction doesn't exist")
    
    df_essential["rxn_ids"].apply(change_bounds)
    



In [17]:
model = load_json_model('./data/Recon3D.json')
model.solver = 'gurobi' #you can use cplex or glpk as you whish, but gurobi and cplex are usually faster

In [20]:
rxn_essential(model)

Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpuzf2assn.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmp0bfay6z6.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpn47tpo9n.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use 

In [4]:
wt_growth = model.slim_optimize()

In [7]:
df_res_rxn = single_reaction_deletion(model, processes=10)


Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmprlje3acw.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpvqk1sa_f.lp
Reading time = 0.03 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use only - expires 2026-11-04
Read LP format model from file /var/folders/0f/7pcrwybx3_zfsnzs2qjgxp8h0000gn/T/tmpf2r0ltg6.lp
Reading time = 0.02 seconds
: 5835 rows, 21200 columns, 80850 nonzeros
Set parameter Username
Set parameter LicenseID to value 2732828
Academic license - for non-commercial use 

In [8]:
df_res_rxn

,ids,growth,status
0,{NDPK10n},752.325899,optimal
1,{EX_mercplaccys_e},755.003216,optimal
2,{IMACTD},755.003216,optimal
3,{GALFUC12GAL14ACGLCGALGLUSIDEtg},755.003216,optimal
4,{HMR_2671},755.003216,optimal
...,...,...,...
10595,{ARGSERSERt},755.003216,optimal
10596,{r0444},755.003216,optimal
10597,{35DHPVSitr},755.003216,optimal
10598,{XOLEST205tl},755.003216,optimal


In [15]:
df_res_rxn.ids=df_res_rxn.ids.apply(lambda x: list(x)[0] if isinstance(x, (set, frozenset)) else x)

In [16]:
df_res_rxn

,ids,growth,status
0,NDPK10n,752.325899,optimal
1,EX_mercplaccys_e,755.003216,optimal
2,IMACTD,755.003216,optimal
3,GALFUC12GAL14ACGLCGALGLUSIDEtg,755.003216,optimal
4,HMR_2671,755.003216,optimal
...,...,...,...
10595,ARGSERSERt,755.003216,optimal
10596,r0444,755.003216,optimal
10597,35DHPVSitr,755.003216,optimal
10598,XOLEST205tl,755.003216,optimal


growth = new biomass flux after the reaction was removed

In [10]:
df_res_rxn[df_res_rxn["status"]!="optimal"]

,ids,growth,status
